In [6]:
import os
from tkinter import *
import tkinter.messagebox
import time
from mutagen.mp3 import MP3
import threading
from tkinter import filedialog
from pygame import mixer
root = Tk()

statusbar = Label(root,text = "Welcome to Music player",bg="#cccccc",relief = SUNKEN,anchor = W)
statusbar.pack(side = BOTTOM, fill = X)

leftframe = Frame(root)
leftframe.pack(side = LEFT,padx=20)

playlistbox = Listbox(leftframe)
playlistbox .pack()

def browse_file():
    global filename_path  
    filename_path = filedialog.askopenfilename()
    add_to_playlist(filename_path)

addBtn = Button(leftframe,text="+Add",command = browse_file)
addBtn.pack(side = LEFT)

def del_song():
    selected_song = playlistbox.curselection()
    selected_song = int(selected_song[0])
    playlistbox.delete(selected_song)
    playlist.pop(selected_song)

delBtn = Button(leftframe,text="-Del",command = del_song)
delBtn.pack(side=LEFT,padx=10)

rightframe = Frame(root)
rightframe.pack()

topframe = Frame(rightframe)
topframe.pack()

lengthlabel = Label(topframe,text = "Total length = --:--",relief = GROOVE,borderwidth=2)
lengthlabel.pack()

currentdurationlabel = Label(topframe,text = "Current time = --:--",relief=GROOVE,borderwidth=2)
currentdurationlabel.pack(pady=10)

mixer.init()  #initializing the mixer 

root.title('Music player')

root.iconbitmap('favicon.ico')



def show_details(play_song):
    file_data = os.path.splitext(play_song)  
    
    if file_data[1] == '.mp3':
        audio = MP3(play_song)
        total_length = audio.info.length
    else:
        a = mixer.Sound(play_song)
        total_length = a.get_length()
        
    mins,secs = divmod(total_length,60)
    mins =round(mins)
    secs = round(secs)
    timeformat = '{:02d}:{:02d}'.format(mins,secs)
    lengthlabel['text'] = "Total length " + " " + timeformat 
    
    t1 = threading.Thread(target = start_count,args = (total_length,))
    t1.start()
    

def start_count(t):
    global paused
    current_time = 0
    while current_time <= t and mixer.music.get_busy():
        if paused:
            continue
        else:
            mins,secs = divmod(current_time,60)
            mins =round(mins)
            secs = round(secs)
            timeformat = '{:02d}:{:02d}'.format(mins,secs)
            currentdurationlabel['text'] = "Current time " + " " + timeformat 
            time.sleep(1)
            current_time += 1

            
#creating a menu bar 
menubar = Menu(root)
root.configure(menu = menubar)

    
playlist = []   #playlist contain the full path and filename  
    
def add_to_playlist(filename):
    filename= os.path.basename(filename)
    index =0
    playlistbox.insert(index,filename)
    playlist.insert(index,filename_path)
    index += 1

#create the submenu
subMenu = Menu(menubar,tearoff =0) #tearoff is used for removing the dashlines from submenu bar
menubar.add_cascade(label="File",menu=subMenu)
subMenu.add_command(label = "Open",command = browse_file)
subMenu.add_command(label = "Exit",command=root.destroy)

def about_us():
    tkinter.messagebox.showinfo('Music player','This music player is built with the help of python tkinter.')

subMenu = Menu(menubar,tearoff =0)
menubar.add_cascade(label="Help",menu=subMenu)
subMenu.add_command(label = "About us",command = about_us)

play = PhotoImage(file = "playbutton.png")
stop = PhotoImage(file = "stop.png")
pause = PhotoImage(file = "pause.png")
rewind = PhotoImage(file = "rewind.png")
fastforward = PhotoImage(file = "fastforward.png")


def play_button():
    global paused
    if paused:
        mixer.music.unpause()
        statusbar['text'] = "Music Resumed"
        paused = FALSE  
    else:
        try:
            selected_song = playlistbox.curselection()
            selected_song = int(selected_song[0])
            play_it = playlist[selected_song]
            mixer.music.load(play_it)
            mixer.music.play()
            show_details(play_it)
            stop_music()
            time.sleep(1)
            statusbar['text'] = "Playing music" + " " + os.path.basename(play_it)
        except:
            mixer.music.play()
            #tkinter.messagebox.showerror('Music player','Add file first')
    

def stop_button(): 
    mixer.music.stop()
    statusbar['text'] = "Music stopped"

paused = FALSE   

def pause_button():
    global paused
    paused = TRUE
    mixer.music.pause()
    statusbar['text'] = "Music paused"
    return

def set_vol(val):
    volume = int(val)/100
    mixer.music.set_volume(volume)
    
def rewind_button():
    play_button()
    statusbar['text'] = "Music Rewinded"
    return

def fastforward_button():
    statusbar['text'] = "Music fast forwarded"
    return

muted = FALSE

def mute_button():
    global muted
    if muted:
        mixer.music.set_volume(0.7)
        scale.set(70)
        soundBtn.configure(image = sound)
        statusbar['text'] = "Music Unmuted"
        muted = FALSE
    else:
        mixer.music.set_volume(0)
        scale.set(0)
        soundBtn.configure(image = mute)
        statusbar['text'] = "Music Muted"
        muted = TRUE
        
middleframe  = Frame(rightframe)
middleframe.pack(padx=10,pady=10)

stopBtn = Button(middleframe,image= stop,bg="#F0F8FF",command=stop_button)
stopBtn.grid(row=1,column=1,padx=10)

playBtn = Button(middleframe,image= play,bg="#F0F8FF",command=play_button)
playBtn.grid(row=1,column=2,padx=10)

pauseBtn = Button(middleframe,image= pause,bg="#F0F8FF",command=pause_button)
pauseBtn.grid(row=1,column=3,padx=10)

rewindBtn = Button(middleframe,image= rewind,bg="#F0F8FF",command=rewind_button)
rewindBtn.grid(row=1,column=0,padx=0)

fastforwardBtn = Button(middleframe,image= fastforward,bg="#F0F8FF",command=fastforward_button)
fastforwardBtn.grid(row=1,column=4,padx=0)

bottomframe = Frame(rightframe)
bottomframe.pack()

mute = PhotoImage(file = "mute.png")
sound = PhotoImage(file = "sound.png")
soundBtn = Button(bottomframe,image= sound,bg="#F0F8FF",command=mute_button)
soundBtn.grid(row=0,column=2)

scale = Scale(bottomframe,from_= 0,to=100,orient = HORIZONTAL,command = set_vol)
scale.set(35)
mixer.music.set_volume(0.35)
scale.grid(row=0,column=3)


def on_closing():
    stop_button()
    root.destroy()

root.protocol("WM_DELETE_WINDOW",on_closing)
root.mainloop()